# PRISM PMF Tutorial

## Introduction to Protein-Ligand Binding Free Energy Calculations

This tutorial demonstrates how to calculate protein-ligand binding free energies using the PRISM PMF module. The workflow includes:

1. System preparation and file organization
2. Steered Molecular Dynamics (SMD) simulation
3. Umbrella sampling at multiple distances
4. WHAM analysis for PMF calculation
5. Results visualization and interpretation

### Prerequisites

- A completed PRISM MD system (from `prism.system().build()`)
- GROMACS installed and configured
- Python packages: numpy, matplotlib, scipy

## 1. Setup and Path Configuration

First, let's set up the environment and define our working paths.

## 1. PMF System Remodeling (Prerequisite)

Before PMF calculations, MD systems must be remodeled for PMF optimization. This creates a PMF-ready system with extended box and proper equilibration.

### Why Remodeling is Required:

1. **Box Extension**: PMF calculations need extended simulation box for ligand pulling
2. **System Optimization**: Remodeled structure is optimized for PMF simulations
3. **Equilibration**: Extended system needs equilibration to relax properly
4. **File Preparation**: Creates proper file structure for PMF workflow

In [ ]:
# First, let's remodel the MD system for PMF calculations
import subprocess
import sys
from pathlib import Path

# Define paths
input_system = "./gromacssim"  # Your original MD system
output_dir = "./pmf_system"    # Where to create PMF-ready system

print("="*60)
print("PMF SYSTEM REMODELING")
print("="*60)

# Method 1: Basic remodeling (creates GMX_PMF_SYSTEM)
print("\nStep 1: Running PMF system remodeling...")

remodel_script = Path("prism/pmf/examples/pmf_remodel.py")
cmd = [
    sys.executable, str(remodel_script),
    "--input", input_system,
    "--output", input_system,  # Create PMF system in same directory
    "--pulling-distance", "2.5"
]

print(f"Running: {' '.join(cmd)}")
result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ PMF system remodeling completed successfully")
    print(f"✓ GMX_PMF_SYSTEM created in: {input_system}")
else:
    print("✗ PMF system remodeling failed")
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)

### Equilibration After Remodeling

The remodeled system requires equilibration to relax the extended box. This can be run locally or on a cluster.

In [ ]:
# Check the generated PMF system structure
pmf_system_dir = Path(input_system) / "GMX_PMF_SYSTEM"

print(f"PMF System Directory: {pmf_system_dir}")

if pmf_system_dir.exists():
    print("\nGenerated files:")
    for file in pmf_system_dir.iterdir():
        if file.is_file():
            size = file.stat().st_size
            print(f"  {file.name}: {size:,} bytes")

    # Check equilibration scripts
    slurm_script = pmf_system_dir / "run_equilibration_slurm.sh"
    local_script = pmf_system_dir / "run_equilibration_local.sh"

    print(f"\nEquilibration scripts:")
    print(f"  Cluster (Slurm): {'✓' if slurm_script.exists() else '✗'} {slurm_script}")
    print(f"  Local execution: {'✓' if local_script.exists() else '✗'} {local_script}")

    print(f"\nNext steps for equilibration:")
    print(f"  1. cd {pmf_system_dir}")
    print(f"  2. sbatch run_equilibration_slurm.sh  # (for cluster)")
    print(f"     OR bash run_equilibration_local.sh  # (for local)")
    print(f"  3. Monitor completion: ls equilibration/npt/npt.gro")
else:
    print("ERROR: PMF system directory not found")
    print("Please ensure remodeling completed successfully")

### Alternative: Test Remodeling with Validation

Use the comprehensive test script to validate the remodeling process.

In [ ]:
## 2. Setup and Path Configuration

Now let's set up the environment for PMF calculations using the remodeled system.

In [ ]:
import sys
from pathlib import Path
import os

# Add PRISM to Python path if needed
prism_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(prism_root))

# Import PRISM PMF module
import prism.pmf as pmf

print("PRISM PMF module loaded successfully")
print(f"Working directory: {Path.cwd()}")
print(f"PRISM root: {prism_root}")

## 3. Define Input and Output Paths

Specify the paths for your **remodeled** PMF system and where to save PMF results.

### Updated Path Structure (After Remodeling):

```
Input (Remodeled PMF System):
./gromacssim/
└── GMX_PMF_SYSTEM/              # ← Use this for PMF calculations
    ├── solv_ions.gro            # Remodeled structure  
    ├── topol.top                # System topology
    ├── equilibration/           # Equilibration results
    │   └── npt.gro       # Final equilibrated structure
    └── PMF_SYSTEM_INFO.txt     # Remodeling metadata

Output (PMF Results):
./pmf_results/
├── analysis/                    # Final PMF analysis
│   ├── pmf.xvg                 # PMF data
│   ├── pmf_curve.png           # PMF plot
│   └── pmf_analysis_report.txt
└── pmf_config_used.yaml        # Configuration record

Working Files (created alongside PMF system):
./gromacssim/GMX_PMF_SYSTEM/
├── pmf_smd/                    # SMD simulation files
└── pmf_umbrella/               # Umbrella sampling windows
```

In [ ]:
# Define paths for PMF calculations
# IMPORTANT: Use the remodeled GMX_PMF_SYSTEM directory
pmf_system_dir = "./gromacssim/GMX_PMF_SYSTEM"  # Remodeled PMF system
output_dir = "./pmf_results"                     # Where to save final results

# Verify PMF system directory exists (after remodeling)
pmf_path = Path(pmf_system_dir)
if not pmf_path.exists():
    print(f"ERROR: PMF system directory not found: {pmf_system_dir}")
    print("Please run PMF system remodeling first (see cells above)")
    print("Expected structure: ./gromacssim/GMX_PMF_SYSTEM/")
else:
    print(f"✓ PMF system found: {pmf_path.absolute()}")
    print(f"✓ Output directory: {Path(output_dir).absolute()}")
    
    # Check required files in PMF system
    required_files = [
        pmf_path / "solv_ions.gro",
        pmf_path / "topol.top",
        pmf_path / "PMF_SYSTEM_INFO.txt"
    ]
    
    print("\nChecking PMF system files:")
    for file in required_files:
        exists = file.exists()
        status = "FOUND" if exists else "MISSING"
        print(f"  [{status}] {file.relative_to(pmf_path)}")
    
    # Check equilibration status
    equilibration_complete = (pmf_path / "equilibration" / "npt.gro").exists()
    eq_status = "COMPLETED" if equilibration_complete else "PENDING"
    print(f"  [STATUS] Equilibration: {eq_status}")
    
    if not equilibration_complete:
        print("\n⚠️  Equilibration not completed yet.")
        print("   Run equilibration before PMF calculations:")
        print(f"   cd {pmf_path}")
        print("   sbatch run_equilibration_slurm.sh")
    else:
        print("\n✅ System ready for PMF calculations!")

## 4. Quick Start: One-Line PMF Calculation

The simplest way to calculate binding free energy using the remodeled system. This runs the complete workflow automatically.

In [ ]:
# Quick start: complete PMF workflow using remodeled system
# Note: This will take several hours to complete

results = pmf.run_pmf_workflow(
    md_system_dir=pmf_system_dir,  # Use remodeled PMF system
    output_dir=output_dir
)

print("\n" + "="*60)
print("PMF CALCULATION COMPLETE")
print("="*60)
print(f"Binding Energy: {results['binding_energy']['value']:.2f} kcal/mol")
print(f"Standard Error: {results['binding_energy']['error']:.2f} kcal/mol")
print(f"\nResults saved in: {output_dir}")

## 4. Standard Workflow: PMFSystem with Custom Parameters

For better control, use PMFSystem with custom pulling distance based on your ligand size.

### Choosing Pulling Distance:
- Small ligands (MW < 300): 1.5-2.0 nm
- Medium ligands (MW 300-500): 2.0-2.5 nm
- Large ligands (MW > 500): 2.5-3.5 nm

In [ ]:
# Create PMF system with custom pulling distance
pulling_distance = 2.5  # nm - adjust based on your ligand size

system = pmf.pmf_system(
    system_dir=md_system_dir,
    output_dir=output_dir,
    pulling_distance=pulling_distance
)

print(f"PMF system created")
print(f"Pulling distance: {pulling_distance} nm")
print(f"\nSystem will be pulled along Z-axis")
print(f"Total Z-extension: {2 * pulling_distance} nm")

### Check Workflow Status

In [ ]:
# Check current workflow status
status = system.get_status()

print("Workflow Status:")
print(f"  Current stage: {status['current_stage']}")
print(f"  Next action: {status['next_action']}")
print(f"  Completed steps: {status.get('completed_steps', [])}")

### Run Complete Workflow

In [ ]:
# Run complete PMF workflow
results = system.run(mode='auto')

print("\n" + "="*60)
print("PMF CALCULATION RESULTS")
print("="*60)
print(f"Binding Free Energy: {results['binding_energy']['value']:.2f} kcal/mol")
print(f"Standard Error: {results['binding_energy']['error']:.2f} kcal/mol")
print(f"\nOutput directory: {output_dir}")

## 5. Step-by-Step Workflow

For manual control and intermediate checking, execute each stage separately.

### Stage 1: Prepare SMD Simulation

This stage:
- Copies essential files from MD system
- Preprocesses structure (remove PBC, center system)
- Creates extended box for pulling
- Generates SMD input files

In [ ]:
# Initialize PMF system
system = pmf.pmf_system(
    system_dir=md_system_dir,
    output_dir=output_dir,
    pulling_distance=2.5
)

# Step 1: Prepare SMD simulation
print("Stage 1: Preparing SMD simulation...")
smd_results = system.build(step='smd')

print("\nSMD Preparation Complete:")
print(f"  SMD directory: {smd_results['smd_dir']}")
print(f"  Input structure: {smd_results.get('input_structure', 'N/A')}")
print(f"  Pull coordinate file: {smd_results.get('pullx_file', 'N/A')}")
print(f"\nTo run SMD manually:")
print(f"  cd {smd_results['smd_dir']}")
print(f"  bash run_smd.sh")

### Stage 2: Analyze SMD Results

After SMD simulation completes, analyze the pulling trajectory.

In [ ]:
# Analyze SMD results (after SMD simulation completes)
print("Analyzing SMD results...")
smd_analysis = system.analyze_smd()

print("\nSMD Analysis:")
print(f"  Maximum force: {smd_analysis.get('max_force', 'N/A'):.2f} kJ/mol/nm")
print(f"  Average force: {smd_analysis.get('avg_force', 'N/A'):.2f} kJ/mol/nm")
print(f"  Total pulling distance: {smd_analysis.get('total_distance', 'N/A'):.2f} nm")
print(f"\nForce profile saved: {smd_analysis.get('force_plot', 'N/A')}")

### Stage 3: Prepare Umbrella Sampling

This stage:
- Analyzes SMD trajectory
- Selects optimal umbrella sampling windows
- Creates input files for each window
- Generates parallel execution scripts

### Window Selection Strategy:
- Close distances (< 1.5 nm): 0.1 nm spacing
- Far distances (> 1.5 nm): 0.2 nm spacing
- Typical total windows: 20-40

In [ ]:
# Step 2: Prepare umbrella sampling
print("Stage 2: Preparing umbrella sampling...")
umbrella_results = system.build_umbrella_step()

print("\nUmbrella Sampling Setup:")
print(f"  Umbrella directory: {umbrella_results['umbrella_dir']}")
print(f"  Number of windows: {umbrella_results['num_windows']}")
print(f"  Window spacing: adaptive (0.1-0.2 nm)")
print(f"\nWindow directories created:")
for i, window in enumerate(umbrella_results.get('windows', [])[:5]):
    print(f"    window_{i:03d}: {window:.2f} nm")
if umbrella_results['num_windows'] > 5:
    print(f"    ... and {umbrella_results['num_windows'] - 5} more windows")

print(f"\nTo run umbrella sampling:")
print(f"  cd {umbrella_results['umbrella_dir']}")
print(f"  bash run_all_umbrella.sh parallel   # Run windows in parallel")
print(f"  bash run_all_umbrella.sh sequential # Run windows sequentially")

### Stage 4: WHAM Analysis and Visualization

After umbrella sampling completes, perform WHAM analysis to calculate PMF.

This stage:
- Collects data from all umbrella windows
- Performs WHAM analysis with bootstrap error estimation
- Calculates binding free energy
- Generates visualization plots

In [ ]:
# Step 3: Perform WHAM analysis
print("Stage 3: Performing WHAM analysis...")
analysis_results = system.build_analysis_step()

print("\n" + "="*60)
print("WHAM ANALYSIS COMPLETE")
print("="*60)
print(f"Binding Free Energy: {analysis_results['binding_energy']['value']:.2f} kcal/mol")
print(f"Standard Error: {analysis_results['binding_energy']['error']:.2f} kcal/mol")

print(f"\nAnalysis directory: {analysis_results['analysis_dir']}")
print(f"\nGenerated files:")
print(f"  PMF data: {analysis_results.get('pmf_file', 'pmf.xvg')}")
print(f"  Error data: {analysis_results.get('error_file', 'pmferror.xvg')}")
print(f"  PMF plot: {analysis_results.get('pmf_plot', 'pmf_curve.png')}")
print(f"  Full report: {analysis_results.get('report_file', 'pmf_analysis_report.txt')}")

## 6. Results Visualization and Analysis

Load and visualize the PMF results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get detailed analysis
pmf_analysis = system.analyze_pmf()

# Extract data
distances = pmf_analysis['pmf_data']['distances']  # nm
pmf_values = pmf_analysis['pmf_data']['pmf']       # kcal/mol
errors = pmf_analysis['pmf_data']['errors']        # kcal/mol

# Plot PMF curve
plt.figure(figsize=(10, 6))
plt.plot(distances, pmf_values, 'b-', linewidth=2, label='PMF')
plt.fill_between(distances, 
                 pmf_values - errors, 
                 pmf_values + errors, 
                 alpha=0.3, label='Error')
plt.xlabel('Distance (nm)', fontsize=12)
plt.ylabel('PMF (kcal/mol)', fontsize=12)
plt.title('Potential of Mean Force', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nPMF Statistics:")
print(f"  Minimum PMF: {np.min(pmf_values):.2f} kcal/mol at {distances[np.argmin(pmf_values)]:.2f} nm")
print(f"  Maximum PMF: {np.max(pmf_values):.2f} kcal/mol at {distances[np.argmax(pmf_values)]:.2f} nm")
print(f"  Binding energy: {pmf_analysis['binding_energy']['value']:.2f} kcal/mol")

### Quality Metrics

Check the quality of PMF calculation.

In [ ]:
# Quality assessment
quality = pmf_analysis.get('quality_metrics', {})

print("PMF Quality Metrics:")
print(f"  Convergence: {quality.get('convergence', 'N/A'):.3f}")
print(f"  Window overlap: {quality.get('window_overlap', 'N/A'):.3f}")
print(f"  Error/Signal ratio: {quality.get('error_ratio', 'N/A'):.3f}")

print("\nQuality Assessment:")
convergence = quality.get('convergence', 0)
if convergence > 0.95:
    print("  Excellent convergence")
elif convergence > 0.90:
    print("  Good convergence")
elif convergence > 0.80:
    print("  Acceptable convergence")
else:
    print("  Poor convergence - consider longer sampling")

overlap = quality.get('window_overlap', 0)
if overlap > 0.15:
    print("  Excellent window overlap")
elif overlap > 0.10:
    print("  Good window overlap")
else:
    print("  Poor window overlap - consider more windows")

## 9. Summary and Best Practices

### Complete PMF Workflow Summary:

1. **System Remodeling** (Prerequisite):
   - Use `pmf_remodel.py` to create PMF-optimized system
   - Run equilibration to relax extended box
   - Validate with `test_pmf_remodeling.py`

2. **PMF Calculation**:
   - Initialize with remodeled system (`GMX_PMF_SYSTEM`)
   - Run workflow (auto or step-by-step)
   - Analyze results and check quality metrics

3. **Results Analysis**:
   - Check convergence and window overlap
   - Generate publication-quality plots
   - Export data for further analysis

### Path Management:

- **Original MD**: `./gromacssim/GMX_PROLIG_MD/` (PRISM output)
- **Remodeled System**: `./gromacssim/GMX_PMF_SYSTEM/` (PMF input)
- **Working files**: Created alongside PMF system
- **Final results**: User-specified output directory

### Remodeling Best Practices:

1. **Always remodel** MD systems before PMF calculations
2. **Complete equilibration** before starting PMF workflow
3. **Test first** with `test_pmf_remodeling.py`
4. **Monitor equilibration** on cluster systems
5. **Verify completion** with `equilibration/npt/npt.gro`

### PMF Best Practices:

1. **Pulling distance**: Choose based on ligand size (1.5-3.5 nm)
2. **Quality check**: Ensure convergence > 0.9 and good window overlap
3. **Error bars**: Keep errors < 1 kcal/mol for reliable results
4. **Sampling time**: Increase if PMF curve is not smooth
5. **Window spacing**: Use finer spacing in binding region

### Configuration Options:

- **Cluster execution**: Use `cluster_128.yaml` for your specific cluster
- **Profile selection**: `quick`, `standard`, `extended`, `debug`
- **Local vs Slurm**: Automatically generated scripts for both

### Typical Timeline:

- **Remodeling**: Minutes (file operations)
- **Equilibration**: 2-12 hours (depending on system size)
- **SMD (5 ns)**: 1-6 hours
- **Umbrella sampling**: Several hours to days
- **WHAM analysis**: Minutes to hours

### Troubleshooting:

**Remodeling Issues**:
- Missing input files: Ensure PRISM MD completed successfully
- Equilibration fails: Check GROMACS setup and cluster configuration

**PMF Issues**:
- High error: Increase umbrella sampling time
- Non-smooth curve: Add more umbrella windows
- No plateau: Increase pulling distance
- Poor convergence: Longer equilibration time

In [ ]:
# Export data to text file
output_file = Path(output_dir) / "pmf_data_export.txt"

data = np.column_stack([distances, pmf_values, errors])
np.savetxt(
    output_file,
    data,
    header="Distance(nm) PMF(kcal/mol) Error(kcal/mol)",
    fmt='%.6f'
)

print(f"Data exported to: {output_file}")
print(f"\nFirst 5 rows:")
print("Distance(nm)  PMF(kcal/mol)  Error(kcal/mol)")
for i in range(min(5, len(distances))):
    print(f"{distances[i]:12.6f}  {pmf_values[i]:13.6f}  {errors[i]:15.6f}")

## 8. Custom Configuration

Create custom configurations for different use cases.

In [ ]:
# Create configuration templates
config_dir = Path("./pmf_configs")
config_dir.mkdir(exist_ok=True)

# Fast configuration for testing
fast_config = config_dir / "fast_pmf_config.yaml"
pmf.create_pmf_config(str(fast_config), template="fast")
print(f"Fast config created: {fast_config}")

# Accurate configuration for publication
accurate_config = config_dir / "accurate_pmf_config.yaml"
pmf.create_pmf_config(str(accurate_config), template="accurate")
print(f"Accurate config created: {accurate_config}")

# Default configuration
default_config = config_dir / "default_pmf_config.yaml"
pmf.create_pmf_config(str(default_config), template="default")
print(f"Default config created: {default_config}")

print("\nConfiguration templates:")
print("  fast: Quick testing (2 ns SMD, 5 ns/window)")
print("  default: Standard production (5 ns SMD, 20 ns/window)")
print("  accurate: High accuracy (10 ns SMD, 50 ns/window)")

### Using Custom Configuration

In [ ]:
# Use custom configuration
system_custom = pmf.pmf_system(
    system_dir=md_system_dir,
    output_dir="./pmf_results_fast",
    config=str(fast_config),  # Use fast template
    pulling_distance=2.0      # Override pulling distance
)

print("Custom PMF system created with fast configuration")
print(f"Configuration file: {fast_config}")
print(f"Output directory: ./pmf_results_fast")

# Export current configuration
export_config = config_dir / "my_current_config.yaml"
system_custom.export_config(str(export_config))
print(f"\nCurrent configuration exported to: {export_config}")

## 9. Summary and Best Practices

### Workflow Summary:

1. **Prepare paths**: Define input MD system and output directories
2. **Create PMF system**: Initialize with appropriate pulling distance
3. **Run workflow**: Either automatic or step-by-step
4. **Analyze results**: Check quality metrics and binding energy
5. **Visualize**: Generate plots for publication

### Path Management:

- Input: MD system directory with `GMX_PROLIG_MD/`
- Working files: Created alongside MD system (`pmf_smd/`, `pmf_umbrella/`)
- Output: Final analysis results in specified output directory

### Best Practices:

1. **Pulling distance**: Choose based on ligand size (1.5-3.5 nm)
2. **Quality check**: Ensure convergence > 0.9 and good window overlap
3. **Error bars**: Keep errors < 1 kcal/mol for reliable results
4. **Sampling time**: Increase if PMF curve is not smooth
5. **Window spacing**: Use finer spacing in binding region

### Typical Timeline:

- SMD (5 ns): 1-6 hours
- Umbrella sampling: Several hours to days
- WHAM analysis: Minutes to hours

### Troubleshooting:

- High error: Increase umbrella sampling time
- Non-smooth curve: Add more umbrella windows
- No plateau: Increase pulling distance
- Poor convergence: Longer equilibration time

## 10. Additional Resources

- Full documentation: `prism/pmf/README.md`
- API reference: Check docstrings in `prism.pmf` module
- More examples: `prism/pmf/examples/` directory
- Configuration guide: `prism/configs/pmf_config.yaml`

For technical issues, check log files in:
- SMD logs: `{md_system_dir}/pmf_smd/smd.log`
- Umbrella logs: `{md_system_dir}/pmf_umbrella/window_XXX/umbrella.log`
- Analysis logs: `{output_dir}/analysis/wham.log`